In [63]:
import pandas as pd
import numpy as np
import sqlite3
import urllib
import datetime
from bs4 import BeautifulSoup
import re

In [2]:
conn = sqlite3.connect('meatbox.db')
cur = conn.cursor()

In [3]:
# 테이블 생성
cur.execute('create table 상품코드정보(기준일자, 상품코드 text, primary key(기준일자, 상품코드))')
cur.execute('create table 상세정보(상품코드 text, 보관방법, 축종, 품목, 가공방법, 원산지, 브랜드, 등급, 배송불가지역 ,배송정책, 판매대상, EST_NO, 포장상태, primary key(상품코드))')
cur.execute('create table 판매자정보(상품코드 text, 대표자, 상호, 전화, 이메일주소, 사업자등록번호, 사업장소재지, primary key(상품코드))')
cur.execute('create table 상품정보(상품코드 text, kg당가격, 가공일, 유통기한, 최대, 최소, 평균, 유통식별번호, primary key(상품코드))')
cur.execute('create table 판매정보(기준일자, 상품코드 text, 구매, 후기, primary key(기준일자, 상품코드))')

In [94]:
# URL 및 Query string을 받아 홈페이지에 접속한 후 그 결과에 대한 객체(soup)를 리턴
def 홈페이지접근(url, params):
    # URL 생성
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    
    # 홈페이지 접속하기
    res = urllib.request.urlopen(url_params)
    html = res.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

In [97]:
# 홈페이지에 등록된 상품코드들을 수집하는 함수
def 상품코드수집():
    # 접속 설정
    url = 'https://www.meatbox.co.kr/fo/product/productListPage.do'
    params = {'categorySeq':10005,'articleCount':'1000'}
    
    # 홈페이지 접속하기
    soup = 홈페이지접근(url, params)

    # 데이터 수집
    기준일자 = str(datetime.date.today())
    상품코드 = soup.find_all('li', {'data-product-seq':re.compile(r'[0-9]+')})
    상품코드정보 = []
    for 코드 in 상품코드:
        상품코드정보.append((기준일자 , 코드.attrs['data-product-seq']))

    # 데이터 프레임 생성
    상품코드리스트 = pd.DataFrame(상품코드정보, columns=['수집일','상품코드'])
    
    return 상품코드리스트

In [182]:
# 상세정보 수집
def 상세정보수집(soup):
    테이블 = soup.find('table', {'class':'section_td'}).find_all('tr')
    상세정보 = {'상품코드':코드}
    for 레코드 in 테이블:
        속성 = 레코드.find('th').text
        내용 = 레코드.find('td').text
        상세정보[속성] = 내용
    return 상세정보

In [247]:
# 판매자정보 수집
def 판매자정보수집(soup):
    테이블 = soup.find('div', {'class':'pop_comm sale_pop'}).find_all('tr')
    판매자정보 = {'상품코드':코드}
    for 레코드 in 테이블:
        속성 = 레코드.find('th').contents[0].replace('\xa0',' ')
        내용 = 레코드.find('td').contents[0].replace('\xa0',' ')
        판매자정보[속성] = 내용
    return 판매자정보

In [195]:
# 판매정보수집
def 판매정보수집(soup):
    테이블 = soup.find('li',{'class':'bott_info'}).find_all('dl')
    판매정보 = {'기준일자':기준일자, '상품코드':코드}
    for 레코드 in 테이블:
        속성 = 레코드.find('dt').contents[0]
        내용 = 레코드.find('em').contents[0]
        판매정보[속성] = 내용
    return 판매정보

In [253]:
# 판매자정보 추가수집
def 사업자정보수집(사업자번호):
    url = 'https://www.ftc.go.kr/bizCommPop.do'
    params = {'wrkr_no':사업자번호}
    soup = 홈페이지접근(url, params)
    테이블 = soup.find('table',{'class','bbs_default view'}).find_all('tr')
    사업자정보 = {}
    for 레코드 in 테이블:
        속성 = 레코드.find('th').text.strip().replace('\xa0',' ')
        내용 = 레코드.find('td').text.strip().replace('\xa0',' ')
        사업자정보[속성] = 내용
    return 사업자정보

In [334]:
# 시세정보 수집
def 시세수집(시세번호):
    url = 'https://www.meatbox.co.kr/fo/sise/siseViewPage.do'
    params = {'siseSeq':시세번호}
    soup = 홈페이지접근(url, params)
    테이블 = soup.find('div',{'id':'table'}).find('table').find_all('tr')
    시세정보 = []
    for 레코드 in 테이블:
        일자 = 레코드.find('td',{'class':'align'}).contents[0].strip()
        요일 = 레코드.find('td',{'class':'align'}).contents[1].text
        시세 = 레코드.find('td',{'class':'dome_sise'}).find('span',{'class':'much'}).text.replace(',','').replace('원','')
        시세 = int(시세)
        시세정보.append((시세번호, 일자, 요일, 시세))

    시세정보 = pd.DataFrame(시세정보, columns=['시세번호','기준일자','요일','시세'])
    return 시세정보

In [ ]:
시세정보 = 시세수집(3932878)

In [362]:
# 수입이력정보수집
def 수입이력정보수집(이력번호):
    url = 'https://www.meatwatch.go.kr/cs/dst/bdi/selectDistbHistInfoDetail.do'
    params = {'searchWrd':이력번호}
    soup = 홈페이지접근(url, params)
    테이블리스트 = soup.find_all('table')
    수입이력정보 = {}
    for 테이블 in 테이블리스트:
        info = 테이블.attrs['summary']
        테이블 = 테이블.find_all('tr')
        for 레코드 in 테이블:
            속성들 = 레코드.find_all('th')
            내용들 = 레코드.find_all('td')
            for i in range(len(속성들)):
                속성 = 속성들[i].text
                내용 = 내용들[i].text
                수입이력정보[속성] = 내용
    return 수입이력정보

In [363]:
수입이력정보수집(800103065450)

{'이력번호': '800103065450',
 '선하증권번호': 'ONEYRICUF7556900',
 '수입쇠고기 품목명': '냉장쇠고기',
 '원산지 국가': '미국',
 '수출국 도축장': 'Cargill Meat Solutions Corporation,EXCEL CORPORATION',
 '수출국 도축일자': '2018-06-27 ~ 2018-07-10',
 '수출국 가공장': 'Cargill Meat Solutions Corporation,EXCEL CORPORATION',
 '수출국 가공일자': '2018-07-05 ~ 2018-07-11',
 '수출업체': 'CARGILL MEAT SOLUTION CORP.',
 '수입업체': '(주)한결엘에스',
 '수입일자': '2018-08-24',
 '부위명(코드)': '안심(BF001),등심(BF002),채끝(BF003),목심(BF004),앞다리(BF005),갈비(BF010)'}

In [335]:
# 시세번호 및 상품정보 수집

In [ ]:
#http://www.ekapepia.com/user/distribution/distDetail.do?nd57433

In [ ]:
# 상품정보 수집